1. Energy consumption section
2. Weather data section
3. Future prediction

In [ ]:
import tensorflow 
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam
from keras import callbacks
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import joblib
from tensorflow import keras

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Functions

In [ ]:
# energy consumption 

def scaleAllData(df):
    allScalers = []
    for column in df.columns:
        df1 = df[column]
        scaler = MinMaxScaler(feature_range=(0,1))
        df[column] = scaler.fit_transform(np.array(df1.values.reshape(-1, 1)))
        allScalers.append(scaler)
    return df, allScalers

In [ ]:
#energy consumption
def saveScalers(colNames, allScalers, directory, location):
    i = 0
    destination = directory +'/Best' + location+ '/' + Scalers'+location+'/'
    for scaler in allScalers:
        scaler_filename = destination + colNames[i] + ".save"
#         print(scaler_filename)
        joblib.dump(scaler, scaler_filename)
        i+=1

In [ ]:
#energy consumption
def df_to_X_y(df, megawattsPos, window_size= 1): #1 day of feature data
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-1):
        row = [ r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][megawattsPos] 
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
#weather data
def df_to_X_y_weather(df, predictValuePos, window_size= 1): #1 day of feature data
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [ r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][predictValuePos] 
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
#weather data

# getting the day cos data for the next 24 hours for prediction
def prepareCircularTimeData(df, colName, howManyHours, timeSteps):
    
    # has last 3 hours of december 31st
    previousDaysData = np.array(df[colName][8757:][0:]).reshape(1,-1)
    previousDaysSin = np.array(df['Day sin'][8757:][0:]).reshape(1,-1)
    previousDaysCos = np.array(df['Day cos'][8757:][0:]).reshape(1,-1)
    previousYearsSin = np.array(df['Year sin'][8757:][0:]).reshape(1,-1)
    previousYearsCos = np.array(df['Year cos'][8757:][0:]).reshape(1,-1)

    previousDaysData = list(previousDaysData)[0].tolist()
    previousDaysSin = list(previousDaysSin)[0].tolist()
    previousDaysCos = list(previousDaysCos)[0].tolist()
    previousYearsSin = list(previousYearsSin)[0].tolist()
    previousYearsCos = list(previousYearsCos)[0].tolist()

    futurePredData = []
    for i in range(timeSteps):
        mem = []
        mem.append(previousDaysData[i])
        mem.append(previousDaysSin[i])
        mem.append(previousDaysCos[i])
        mem.append(previousYearsSin[i])
        mem.append(previousYearsCos[i])
        futurePredData.append(mem)
    
    # Day sin, Day cos, Year sin, Year cos
    # Need these in addition to the weather data variable information for circular time
    startPos = 0
    endPos = 24

    previousDaysSin = np.array(df['Day sin'][startPos:endPos][0:]).reshape(1,-1)
    previousDaysCos = np.array(df['Day cos'][startPos:endPos][0:]).reshape(1,-1)
    previousYearsSin = np.array(df['Year sin'][startPos:endPos][0:]).reshape(1,-1)
    previousYearsCos = np.array(df['Year cos'][startPos:endPos][0:]).reshape(1,-1)

    previousDaysSin = list(previousDaysSin)[0].tolist()
    previousDaysCos = list(previousDaysCos)[0].tolist()
    previousYearsSin = list(previousYearsSin)[0].tolist()
    previousYearsCos = list(previousYearsCos)[0].tolist()

    for i in range(endPos):
        mem = []
        mem.append(previousDaysSin[i])
        mem.append(previousDaysCos[i])
        mem.append(previousYearsSin[i])
        mem.append(previousYearsCos[i])
        futurePredData.append(mem)
    
#     print(np.array(futurePredData))
    return futurePredData
    

In [ ]:
#weather data 
def futurePredictions_weather(model, data, hourPred, timesteps, numVars):
    future_outputs=[]
    i=0
    beg = 0
    end = 3
    while(i < hourPred): #for the next however many hours
        oneStep = np.array(list(data[beg:end])).reshape(1, timesteps, numVars) #1 list, each index has n_steps (3) of numVars (5) data
        yhat = model.predict(oneStep, verbose=0)#.flatten()
        future_outputs.append(yhat[0][0])
        data[end].insert(0, yhat[0][0])
        beg += 1
        end += 1
        i+=1

    return future_outputs

In [ ]:
def futurePredictions_all(model, lastDay_input, weatherData, hourPred, timesteps, numVars):
    future_outputs=[]
    i=0
    counter = 0
    while(i<hourPred): #for the next thirty hours
        if i == 0: #for the first new predicted value, using old weather data to predict new value 
            lastDay_input = np.array(lastDay_input)
            lastDay_input = lastDay_input.reshape(1, timesteps, numVars)
            yhat = model.predict(lastDay_input, verbose=0)#.flatten()
            print(yhat[0])
#             future_outputs.extend(yhat.tolist())
            future_outputs.append(yhat[0][0])
            i=i+1
        else:# then take from the 1 position and getting the new input and putting it into input
#             print(future_outputs[counter])
            setVal = future_outputs[counter].item()
            weatherData.at[counter, -1:] = setVal #set the megawatts val from the prev predicted
            oneData = np.array(weatherData.iloc[counter, :]) #one day of data 
#             print("{} hour input {}".format(1, oneData))
            oneData = oneData.reshape((1, timesteps, numVars)) #works
            yhat = model.predict(oneData, verbose=0)
#             print("{} hour output {}".format(i,yhat))
#             future_outputs.extend(yhat.tolist())
            future_outputs.append(yhat[0][0])
            i=i+1
            counter+=1

    return future_outputs

In [ ]:
def prepareDf(columns, df, directory, location):

    df['date'] = pd.to_datetime(df['date'], format='%m-%dT%H:%M:%S')
    df['date']= df["date"].map(lambda x: x.replace(year=2019))
    df.index = df['date']
    df.drop('date', axis=1, inplace=True)

    ################################################ preprocess #########################################################
    df, allScalers = scaleAllData(df)
    colNames = df.columns
    
    saveScalers(colNames, allScalers, directory, location) #saving scalers
    
    df['Seconds'] = df.index.map(pd.Timestamp.timestamp)
    day = 60 * 60 * 24 #60 s in a min, 60 min in an hour, 24 hours in a day
    year = 365.2425*day #num seconds in a year
    df['Day sin'] = np.sin(df['Seconds']*2*(np.pi/day))
    df['Day cos'] = np.cos(df['Seconds']*2*(np.pi/day))
    df['Year sin'] = np.sin(df['Seconds']*2*(np.pi/year))
    df['Year cos'] = np.cos(df['Seconds']*2*(np.pi/year))

    df = df.drop('Seconds', axis = 1)

    cols = columns
    df = df[cols]
    return df

In [ ]:
def energyConsumptionModel(df, numEpochs, directory, location):
    megawattsPos = 30
    X, y = df_to_X_y(df, megawattsPos, 1) 
    #############################################train and test #########################################################
    train_percent = 0.9
    validation_percent = 0.1

    train_percent_split = int(train_percent * len(X))
    validation_percent_split = int(validation_percent * len(X)) + train_percent_split
    print(validation_percent_split)

    X_train, y_train = X[:train_percent_split], y[:train_percent_split]
    X_val, y_val = X[train_percent_split:validation_percent_split], y[train_percent_split:validation_percent_split] #validation

    # just gonna have it hold like 1 day of prev data 
    testData = 8760 - 2
    X_test, y_test = X[testData:], y[testData:]

    # X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

    ############################################# create model #########################################################
    model = Sequential()
    model.add(LSTM(64,input_shape=(X_train.shape[1],X_train.shape[2]))) # input layer; 31 data inputs, 1 step (1 hr)
    model.add(Dense(32, 'relu')) #hidden layer 1
    model.add(Dense(32, 'relu')) #hidden layer 2
    model.add(Dense(1, 'linear')) #output layer
    
    storeModel = directory + 'model' + location +'/'
    cp = ModelCheckpoint(storeModel, save_best_only = True) #I want to only save the best model (lowest validation loss)
    model.compile(loss = MeanSquaredError(), optimizer = Adam(), metrics = [RootMeanSquaredError()])

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = numEpochs, callbacks = [cp])

    train_predictions = model.predict(X_train).flatten()
    train_results = pd.DataFrame(data = {'Train Predictions': train_predictions, 'Actuals':y_train})
    rmse = mean_squared_error(train_predictions, y_train, squared=False)
    print("RMSE value: ", rmse)
    return rmse, model

In [ ]:
def getBestEnergyConsumptionModel(iters, df, numEpochs, directory, location):
    getBest = {}
    for i in range(iters):
        rmse, model = energyConsumptionModel(df, numEpochs, directory, location)
        getBest.update({rmse:model})
    lowestRmse = min(getBest.keys())
    return getBest[lowestRmse]
    

In [ ]:
def weatherVariableModel(df, window_size, column, numEpochs, directory, location):
    ################### get a variable to predict for ##################
    getCols = ['Day sin', 'Day cos', 'Year sin', 'Year cos']
    getCols.append(column)
    col = df[getCols]
    predictColPos = len(col.columns) - 1
    X1, y1 = df_to_X_y_weather(col, predictColPos, window_size)

    ################## split data ##################
    train_percent = 0.9
    validation_percent = 0.1
    train_percent_split = int(train_percent * len(X1))
    validation_percent_split = int(validation_percent * len(X1)) + train_percent_split

    X_train1, y_train1 = X1[:train_percent_split], y1[:train_percent_split]
    X_val1, y_val1 = X1[train_percent_split:validation_percent_split], y1[train_percent_split:validation_percent_split] #validation

    # just gonna have it hold like 1 day of prev data 
    testData = validation_percent_split - 2
    X_test1, y_tes1t = X1[testData:], y1[testData:]

    numDataInputs = X_train1.shape[1]
    numSteps = X_train1.shape[2]
    
    ################## Model Creation ##################
    modelx = Sequential()
    modelx.add(LSTM(64,input_shape=(numDataInputs, numSteps))) # input layer; 5 data inputs, 3 step (3 hr)
    modelx.add(Dense(32, 'relu')) #hidden layer 1
    modelx.add(Dense(32, 'relu')) #hidden layer 2
    modelx.add(Dense(1, 'linear')) #output layer
    storeModel = directory + 'modelx' + location + '/'
    cpx = ModelCheckpoint(storeModel, save_best_only = True) #I want to only save the best model (lowest validation loss)
    modelx.compile(loss = MeanSquaredError(), optimizer = Adam(), metrics = [RootMeanSquaredError()])

    modelx.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs = numEpochs, callbacks = [cpx])
    train_predictions = modelx.predict(X_train1).flatten()
    rmse = mean_squared_error(train_predictions, y_train1, squared=False)
    print("RMSE value: ", rmse)
    return rmse, modelx
    

In [ ]:
def getBestWeatherModels(df, window_size, columns, numEpochs, iters, directory, location):
    getBest = []
    for column in columns:
        getBestOneVar = {}
        for i in range(iters):
            rmse, model = weatherVariableModel(df, window_size, column, numEpochs, directory, location)
            getBestOneVar.update({rmse:model})
        #now store the best one for that variables
        lowestRmse = min(getBestOneVar.keys())
        getBest.append(getBestOneVar[lowestRmse])
    return getBest

In [ ]:
def bestPerformingModel(howManyFutureHours, window_size, numEpochs, iters, columnsForDf, columns, df, directory, location):
    df = prepareDf(columnsForDf, df, directory, location)
    bestEnergyConsumptionModel = getBestEnergyConsumptionModel(iters, df, numEpochs, directory, location)
    directory = 'Best'+ location + '/'
    bestEnergyConsumptionModel.save(directory+'energyModel/', save_format = 'utf')
    
    print("--------------------------------- SAVED BEST ENERGY CONSUMPTION MODEL -----------------------------------")
    #we only want weather variable prediction, so only pass the correct stuff
    
    bestWeatherDataModels = getBestWeatherModels(df, window_size, columns, numEpochs, iters, directory, location)
    
    #save the models
    j = 0
    for weatherModel in bestWeatherDataModels:
        weatherModel.save(directory+columns[j], save_format = 'utf')
        j+=1
    
    i = 0
    numVars = 5 # 3 time steps, 5 variables (for weather variable prediction)
    allVarFuturePreds = []
    for column in columns:
        futurePredData = prepareCircularTimeData(df, column, howManyFutureHours, window_size)
        future_outputs = futurePredictions_weather(bestWeatherDataModels[i], futurePredData, howManyFutureHours, window_size, numVars)
        allVarFuturePreds.append(future_outputs)
        i+=1

    df2 = pd.DataFrame(allVarFuturePreds)
    df2 = df2.T
    df2.columns = columns
    # df2.to_csv('weatherPred.csv', encoding='utf-8', index = False) #MIGHT HAVE TO CHANGE THIS TO weatherPred + i

    ########################################## FUTURE PREDICTION #######################################################
    weatherPred = df2
    daySin = df['Day sin'][0:24]
    dayCos = df['Day cos'][0:24]
    yearSin = df['Year sin'][0:24]
    yearCos = df['Year cos'][0:24]

    addFiles = [daySin, dayCos, yearSin, yearCos]
    weatherPred['Day sin'] = daySin.values
    weatherPred['Day cos'] = dayCos.values
    weatherPred['Year sin'] = yearSin.values
    weatherPred['Year cos'] = yearCos.values
    weatherPred['Megawatts_2020'] = 0

    lastDay = df[-1:]
    lst_output = futurePredictions_all(bestEnergyConsumptionModel, lastDay, weatherPred, 24, 1, 31)

    ########################################### check output #######################################################
    scaler_filename = directory + 'Scalers' + location+'/Megawatts_2019.save'
    megawattsScaler = joblib.load(scaler_filename)

    predictedVals = megawattsScaler.inverse_transform(np.array(lst_output).reshape(-1,1))
    predicted24Hours = pd.DataFrame(predictedVals)

    check_directory = '/content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/NOAA_and_CAISO_Data_2020/'
    compFiles = [f for f in listdir(check_directory) if isfile(join(check_directory, f))] 
    flies = []
    for file in compFiles:
    #     print(file)
        flies.append(check_directory + file)
    df1 = pd.read_csv(flies[2])
    actual24Hours = df1['Megawatts_2020'][:24]

    print("----------------------------FINAL, DONE;----------------------------")
    plt.plot(actual24Hours, color = 'blue')
    plt.plot(predicted24Hours, color = 'red')
    plt.show()

    rmse = mean_squared_error(actual24Hours, predicted24Hours, squared=False)
    print("RMSE value: ", rmse)

# Testing

In [ ]:
directory = '/content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/NOAA_and_CAISO_Data_ONEYEAR/'
dataFiles = [f for f in listdir(directory) if isfile(join(directory, f))] 
for file in dataFiles:
    print(file)
    df = pd.read_csv(directory + file)

#for now, I will only use SF data 
fileRead = directory + dataFiles[3]
print('fileRead: ', fileRead)
df = pd.read_csv(fileRead)

SD-NOAA-and-Megawatts.csv
SAC-NOAA-and-Megawatts.csv
LA-NOAA-and-Megawatts.csv
SF-NOAA-and-Megawatts.csv
fileRead:  /content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/NOAA_and_CAISO_Data_ONEYEAR/SF-NOAA-and-Megawatts.csv


In [ ]:
df. columns

Index(['date', 'HLY_TEMP_NORMAL_SF', 'HLY_TEMP_10PCTL_SF',
       'HLY_TEMP_90PCTL_SF', 'HLY_DEWP_NORMAL_SF', 'HLY_DEWP_10PCTL_SF',
       'HLY_DEWP_90PCTL_SF', 'HLY_PRES_NORMAL_SF', 'HLY_PRES_10PCTL_SF',
       'HLY_PRES_90PCTL_SF', 'HLY_CLDH_NORMAL_SF', 'HLY_HTDH_NORMAL_SF',
       'HLY_CLOD_PCTCLR_SF', 'HLY_CLOD_PCTFEW_SF', 'HLY_CLOD_PCTSCT_SF',
       'HLY_CLOD_PCTBKN_SF', 'HLY_CLOD_PCTOVC_SF', 'HLY_HIDX_NORMAL_SF',
       'HLY_WCHL_NORMAL_SF', 'HLY_WIND_AVGSPD_SF', 'HLY_WIND_PCTCLM_SF',
       'HLY_WIND_VCTDIR_SF', 'HLY_WIND_VCTSPD_SF', 'HLY_WIND_1STDIR_SF',
       'HLY_WIND_1STPCT_SF', 'HLY_WIND_2NDDIR_SF', 'HLY_WIND_2NDPCT_SF',
       'Megawatts_2019'],
      dtype='object')

In [ ]:
howManyFutureHours = 24
window_size = 3
numEpochs = 20
iters = 5
columnsForDf = ['HLY_TEMP_NORMAL_SF', 'HLY_TEMP_10PCTL_SF',
       'HLY_TEMP_90PCTL_SF', 'HLY_DEWP_NORMAL_SF', 'HLY_DEWP_10PCTL_SF',
       'HLY_DEWP_90PCTL_SF', 'HLY_PRES_NORMAL_SF', 'HLY_PRES_10PCTL_SF',
       'HLY_PRES_90PCTL_SF', 'HLY_CLDH_NORMAL_SF', 'HLY_HTDH_NORMAL_SF',
       'HLY_CLOD_PCTCLR_SF', 'HLY_CLOD_PCTFEW_SF', 'HLY_CLOD_PCTSCT_SF',
       'HLY_CLOD_PCTBKN_SF', 'HLY_CLOD_PCTOVC_SF', 'HLY_HIDX_NORMAL_SF',
       'HLY_WCHL_NORMAL_SF', 'HLY_WIND_AVGSPD_SF', 'HLY_WIND_PCTCLM_SF',
       'HLY_WIND_VCTDIR_SF', 'HLY_WIND_VCTSPD_SF', 'HLY_WIND_1STDIR_SF',
       'HLY_WIND_1STPCT_SF', 'HLY_WIND_2NDDIR_SF', 'HLY_WIND_2NDPCT_SF',
        'Day sin', 'Day cos', 'Year sin', 'Year cos','Megawatts_2019']

columns = ['HLY_TEMP_NORMAL_SF', 'HLY_TEMP_10PCTL_SF',
       'HLY_TEMP_90PCTL_SF', 'HLY_DEWP_NORMAL_SF', 'HLY_DEWP_10PCTL_SF',
       'HLY_DEWP_90PCTL_SF', 'HLY_PRES_NORMAL_SF', 'HLY_PRES_10PCTL_SF',
       'HLY_PRES_90PCTL_SF', 'HLY_CLDH_NORMAL_SF', 'HLY_HTDH_NORMAL_SF',
       'HLY_CLOD_PCTCLR_SF', 'HLY_CLOD_PCTFEW_SF', 'HLY_CLOD_PCTSCT_SF',
       'HLY_CLOD_PCTBKN_SF', 'HLY_CLOD_PCTOVC_SF', 'HLY_HIDX_NORMAL_SF',
       'HLY_WCHL_NORMAL_SF', 'HLY_WIND_AVGSPD_SF', 'HLY_WIND_PCTCLM_SF',
       'HLY_WIND_VCTDIR_SF', 'HLY_WIND_VCTSPD_SF', 'HLY_WIND_1STDIR_SF',
       'HLY_WIND_1STPCT_SF', 'HLY_WIND_2NDDIR_SF', 'HLY_WIND_2NDPCT_SF']

currentDirectory = '/content/drive/MyDrive/Fairness ML/FairnessML_git/LSTM-BasicModel/'
location = '_SF'

##### MAKE SURE YOU CHANGED ALL VARIABLES RELATED TO LOCATION ABOVE BEFORE RUNNING THIS CELL
bestPerformingModel(howManyFutureHours, window_size, numEpochs, iters, columnsForDf, columns, df, currentDirectory, location)

8758
Epoch 1/20
245/247 [============================>.] - ETA: 0s - loss: 0.0156 - root_mean_squared_error: 0.1250

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Fairness ML/FairnessML_git/LSTM-BasicModel/model_SF/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Fairness ML/FairnessML_git/LSTM-BasicModel/model_SF/assets


247/247 [==============================] - 15s 55ms/step - loss: 0.0156 - root_mean_squared_error: 0.1247 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0424
Epoch 2/20
237/247 [===========================>..] - ETA: 0s - loss: 0.0019 - root_mean_squared_error: 0.0435

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Fairness ML/FairnessML_git/LSTM-BasicModel/model_SF/assets


...

INFO:tensorflow:Assets written to: Best_SF/HLY_WIND_2NDPCT_SF/assets


INFO:tensorflow:Assets written to: Best_SF/HLY_WIND_2NDPCT_SF/assets


[0.4644548]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


FileNotFoundError: ignored